In [1]:
pip install pandasql


Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install langchain langchain-core langchain-community langchain-google-genai google-generativeai


In [3]:
# Importing Libraries
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import create_sql_query_chain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableMap
from langchain_google_genai import ChatGoogleGenerativeAI

import warnings
warnings.filterwarnings('ignore')



In [4]:
#Connect SQL Database
!pip install pyodbc pandas sqlalchemy


In [5]:
import pyodbc

# Connection string
conn_str = (
    r'DRIVER={ODBC Driver 17 for SQL Server};'
    r'SERVER=DESKTOP-KL2GSS2;'
    r'DATABASE=Amazon_sales;'
    r'Trusted_Connection=yes;'
)

# Connect
conn = pyodbc.connect(conn_str)
print("✅ Connected to SQL Server successfully!")


✅ Connected to SQL Server successfully!


In [6]:
import pandas as pd

query = "SELECT  * FROM  SalesData"
df = pd.read_sql(query, conn)

df.head()


,Transaction_ID,Date,Customer_ID,Gender,Age,Product_Category,Quantity,Price_per_Unit,Total_Amount
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction_ID    1000 non-null   int64 
 1   Date              1000 non-null   object
 2   Customer_ID       1000 non-null   object
 3   Gender            1000 non-null   object
 4   Age               1000 non-null   int64 
 5   Product_Category  1000 non-null   object
 6   Quantity          1000 non-null   int64 
 7   Price_per_Unit    1000 non-null   int64 
 8   Total_Amount      1000 non-null   int64 
dtypes: int64(5), object(4)
memory usage: 70.4+ KB


In [8]:
llm = ChatGoogleGenerativeAI(
    model = 'gemini-2.0-flash',
    api_key = 'your_secret_api_key'
)

In [9]:
# Create the LLM prompt template

from langchain_core.prompts import ChatPromptTemplate

template = """ Based on the table schema below, write a SQL query that would answer the user's question : 
Remember: Only provide me the SQL query don't include anything else. Provide me SQL query in a single line dont add line breaks.
Schema: {Schema}
Question: {question}
"""
prompt= ChatPromptTemplate.from_template(template)

In [10]:
# Get the schema of database

def get_schema(df_name="SalesData"):
    schema_str = f"""
Table: {df_name}
Columns:
- Transaction_ID (int)
- Date (string)
- Customer_ID (string)
- Gender (string)
- Age (int)
- Product_Category (string)
- Quantity (int)
- Price_per_Unit (int)
- Total_Amount (int)
"""
    return schema_str.strip()

In [11]:
# Create the SQL query chain using the LLM and the prompt template

sql_chain = (
    RunnableMap(
        { "Schema": lambda inputs: get_schema(df),         # Inject Schema automatically from your dataframe
           "question": lambda inputs: inputs["question"]      # Take question from user input
        }
    )
    | prompt  # your ChatPromptTemplate
    | llm.bind(stop=["\nSQLResult:"])  # stop token
    | StrOutputParser()  # convert LLM output to string
)

In [12]:
# Test the SQL query chain with a sample question 

resp = sql_chain.invoke({"question": "Who did most shopping between male and female and also include count?"})

resp = print(resp)


```sql
SELECT Gender, COUNT(*) AS PurchaseCount FROM transactions GROUP BY Gender ORDER BY PurchaseCount DESC LIMIT 1;
```


In [13]:
resp = """```sql
SELECT Gender, COUNT(*) AS PurchaseCount FROM transactions GROUP BY Gender ORDER BY PurchaseCount DESC LIMIT 1;
```"""

In [14]:
import re

query = re.search(r"```sql\s*(.*?)\s*```", resp, re.DOTALL | re.IGNORECASE)


if query:
    query = query.group(1).strip()
    print(query)
else:
    print("No SQL query found.")

SELECT Gender, COUNT(*) AS PurchaseCount FROM transactions GROUP BY Gender ORDER BY PurchaseCount DESC LIMIT 1;


In [15]:
 # Run SQL query on Pandas DataFrame `df`
from pandasql import sqldf

In [16]:
# Step 3: Fix table name (replace anything with df)
# This makes sure your SQL runs on your DataFrame
query = re.sub(r"FROM\s+\w+", "FROM df", query, flags=re.IGNORECASE)

print("\n Modified SQL Query (using df):\n", query)


 Modified SQL Query (using df):
 SELECT Gender, COUNT(*) AS PurchaseCount FROM df GROUP BY Gender ORDER BY PurchaseCount DESC LIMIT 1;


In [17]:
# define a helper function for SQL-on-Pandas
pysqldf = lambda q: sqldf(q, globals())

In [18]:
try:
    result = pysqldf(query)
    print("\n Query Result:")
    display(result)
except Exception as e:
    print("\nError while executing SQL on DataFrame:", e)


 Query Result:


,Gender,PurchaseCount
0,Female,510
